In [2]:
import pandas as pd
import numpy as np
import os

In [4]:
DATA_PATH = '/home/chrisams/Documents/data_TAIM/raw'
COLUMNS = [
    'Fecha Hora (YYYY-MM-DD HH:MM)',
    'Velocidad de viento en 20.0 metros [mean,m/s]',
    'Velocidad de viento en 10.0 metros [mean,m/s]',
]
V20 = 'Velocidad de viento en 20.0 metros [mean,m/s]'
V10 = 'Velocidad de viento en 10.0 metros [mean,m/s]'

In [ ]:
# Read all the data files.
for path, _, file_names in os.walk(DATA_PATH):
    eolic_plants_df = [None] * len(file_names)
    for i, file_name in enumerate(file_names):
        full_path = os.path.join(path, file_name)
        eolic_plants_df[i] = pd.read_csv(full_path)

In [ ]:
# Clean data and keep only useful columns and rows.

min_date = None
max_date = None

for i in range(len(eolic_plants_df)):
    plant =  eolic_plants_df[i]
    plant = plant[COLUMNS].set_index(\
                pd.DatetimeIndex(plant['Fecha Hora (YYYY-MM-DD HH:MM)'])\
            ).iloc[:, 1:]
    v10_mean = plant[V10].mean()
    v20_mean = plant[V20].mean()
    alpha = np.log(v10_mean / v20_mean) / np.log(10 / 20)
    
    plant['85m_speed'] = plant[V20] * np.power(85 / 20, alpha)
    
    current_min_date = plant.index.min()
    current_max_date = plant.index.max()
    
    if min_date:
        if min_date < current_min_date:
            min_date = current_min_date
    else:
        min_date = current_min_date
    
    if max_date:
        if max_date > current_max_date:
            max_date = current_max_date
    else:
        max_date = current_max_date

    eolic_plants_df[i] = plant['85m_speed']

for i in range(len(eolic_plants_df)):
    eolic_plants_df[i] = eolic_plants_df[i][min_date:max_date]

In [ ]:
for i, plant in enumerate(eolic_plants_df):
    plant.to_csv(os.path.join(DATA_PATH, 'plant_{}'.format(i)))

In [5]:
plant1 = pd.read_csv(os.path.join(DATA_PATH, 'plant_1'), index_col=0, names=['85m_speed'], parse_dates=True)

In [8]:
DATA_PATH = '/home/chrisams/Documents/data_TAIM/processed'

In [9]:
for path, _, file_names in os.walk(DATA_PATH):
    for file_name in file_names:
        t = pd.read_csv(os.path.join(DATA_PATH, 'plant_1'), index_col=0, names=['85m_speed'], parse_dates=True)
        print(t.shape)
        

(398284, 1)
(398284, 1)
(398284, 1)
(398284, 1)
(398284, 1)
(398284, 1)
(398284, 1)
